In [ ]:
import numpy as np
import scanpy as sc
import os
import pandas as pd
import doubletdetection

In [ ]:
# env: cyno_env_local

doublet detection algorithm according to https://doubletdetection.readthedocs.io/en/latest/tutorial.html

In [ ]:
pre = "H07_01"
drive = "F"
base_package_version_path = os.path.join(drive + ":\\","monkey_IZI","analysis","package_versions")
base_table_path = os.path.join(drive + ":\\","monkey_IZI","analysis","tables",pre)
idents = ['Human1_6hr_S3','Human1_24hr_S5','Human1_TimeZero_S1','Human2_6hr_S4','Human2_24hr_S6','Human2_TimeZero_S2']

run doubletdetection separately for each sample

In [ ]:
for ident in idents:
    path_filtered_counts = os.path.join(drive + ":\\","monkey_IZI","nextflow","outdir_human_ensemble","human","cellranger","count",ident,"outs","filtered_feature_bc_matrix.h5")
    adata_filtered_bc = sc.read_10x_h5(path_filtered_counts)
    adata_filtered_bc.var_names_make_unique()
    sc.pp.filter_genes(adata_filtered_bc, min_cells=1)
    
    clf = doubletdetection.BoostClassifier(
        n_iters=10,
        clustering_algorithm="louvain",
        standard_scaling=True,
        pseudocount=0.1,
        n_jobs=-1,
    )
    doublets = clf.fit(adata_filtered_bc.X).predict(p_thresh=1e-16, voter_thresh=0.5)
    doublet_score = clf.doublet_score()
    
    adata_filtered_bc.obs["doublet"] = doublets.copy()
    adata_filtered_bc.obs["doublet_score"] = doublet_score.copy()
    adata_filtered_bc.obs['sample'] = ident
    adata_filtered_bc.obs.to_csv(os.path.join(base_table_path,ident + '_DT_doub_pred.csv'))

save session infos

In [ ]:
sc.logging.print_header()

In [ ]:
import pkg_resources
with open(os.path.join(base_package_version_path, pre + '_package_versions.txt'), "w") as file:
    for package in pkg_resources.working_set:
        file.write(f"{package.key}=={package.version}\n")
        print(f"{package.key}=={package.version}")